In [ ]:
#@title Установка необходимых библиотек
%pip install --quiet --upgrade diffusers accelerate mediapy triton scipy ftfy spacy==3.4.4
%pip install transformers
# Пакет xformers обязателен для создания нескольких изображений размером 768x768
%pip install -q xformers==0.0.16rc425

In [ ]:
#@title Установка и настройка модели
import mediapy as media
import torch
from diffusers import StableDiffusionPipeline
import random
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler, DPMSolverMultistepScheduler

# Настройки модели Diffusion
scheduler = None
model_id = "stabilityai/stable-diffusion-2-1"
device = "cuda"

if model_id.startswith("stabilityai/"):
  model_revision = "fp16"
else:
  model_revision = None

if scheduler is None:
  pipe = StableDiffusionPipeline.from_pretrained(
      model_id,
      torch_dtype=torch.float16,
      revision=model_revision,
      )
else:
  pipe = StableDiffusionPipeline.from_pretrained(
      model_id,
      scheduler=scheduler,
      torch_dtype=torch.float16,
      revision=model_revision,
      )

pipe = pipe.to(device)
pipe.enable_xformers_memory_efficient_attention()

if model_id.endswith('-base'):
  image_length = 512
else:
  image_length = 768

In [ ]:
#@title Настройте параметры дома
budget = "$$" #@param ["$", "$$", "$$$"]
size = "medium" #@param ["compact", "medium", "large"]
number_of_stories = "two-story" #@param ["one-story", "two-story", "three-story", "split-level"]
siding_material = "brick" #@param ["mixed siding", "wood", "stucco", "brick"]
siding_color = "multi" #@param ["white", "beige", "gray", "brown", "multi", "black", "red", "yellow"]
roof_type = "hip" #@param ["gable", "hip", "shed", "flat", "clipped gable"]
roof_color = "brown" #@param ["gray", "black", "brown", "red"]
additional_requests = "" #@param {type:"string"}
number_of_images = "4" #@param ["2", "4", "6", "8"]
data = {
    "id": "474305464",
    "budget": budget,
    "size": size,
    "number of stories": number_of_stories,
    "siding material": siding_material,
    "siding color": siding_color,
    "roof type": roof_type,
    "roof color": roof_color,
    "image": ""
}

In [ ]:
#@title Формирование промпта и генерация изображений
# Формирование промпта
prompt = "Generate an image of a house with the following specifications:"

for key, value in data.items():
    if key != "id" and key != "image":
        prompt += f"\n{key.capitalize()}: {value}"

if additional_requests != "":
    prompt += additional_requests

prompt = prompt
remove_safety = False
num_images = int(number_of_images)
seed = random.randint(1, 999999)
pipe.safety_checker = None

images = pipe(
    prompt,
    height = image_length,
    width = image_length,
    num_inference_steps = 25,
    guidance_scale = 9,
    num_images_per_prompt = num_images,
    generator = torch.Generator("cuda").manual_seed(seed)
    ).images

media.show_images(images)
display(f"Seed: {seed}")
images[0].save("output.jpg")